In [6]:
import os
import numpy as np
import pandas as pd

from mpire import WorkerPool

REF_PATH = "/data1/zhoujb/cowpeaSequence/refernce/"
RAW_SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"

In [2]:
"""
将参考基因组变成Pos
"""
chr_name_map = {"CM072825.1":"Chr01", "CM072826.1":"Chr02", "CM072827.1":"Chr03", 
                "CM072828.1":"Chr04", "CM072829.1":"Chr05", "CM072830.1":"Chr06", 
                "CM072831.1":"Chr07", "CM072832.1":"Chr08", "CM072833.1":"Chr09", 
                "CM072834.1":"Chr10", "CM072835.1":"Chr11"}

seq = ""
with open(os.path.join(REF_PATH, "G98.fna")) as f:
    for line in f:
        line = line.rstrip()
        if (line.startswith(">")) and (seq == ""):
            header = line.split(" ")[0][1:]
        elif line.startswith(">"):
            if header in chr_name_map:
                with open(os.path.join(REF_PATH, "chrPos/{}.pos".format(chr_name_map[header])), "w") as out_f:
                    for pos, val in enumerate(list(seq)):
                        print("\t".join([str(pos+1), val.upper()]), file=out_f)
                
                print("{} DONE".format(chr_name_map[header]))
                
                seq = ""
                header = line.split(" ")[0][1:]
        else:
            seq += line

print("ALL DONE")

Chr01 DONE
Chr02 DONE
Chr03 DONE
Chr04 DONE
Chr05 DONE
Chr06 DONE
Chr07 DONE
Chr08 DONE
Chr09 DONE
Chr10 DONE
Chr11 DONE
ALL DONE


In [11]:
# 将所有区间对应Pos写成文件
def getRefPos(file_name):
    raw_snp = pd.read_table(os.path.join(RAW_SNP_PATH, file_name), usecols=['Pos', 'Chr'], index_col="Pos")
    ref_file = pd.read_table(os.path.join(REF_PATH, "chrPos/{}.pos".format(file_name.split("_")[0])), 
                                          header=None, names=["Pos", "Val"], index_col="Pos")
    ref_file = ref_file.loc[raw_snp.index]
    ref_file.to_csv(os.path.join(REF_PATH, "cowpeaRef/{}.pos".format(file_name.split(".")[0])), sep="\t")
    return

if __name__ == "__main__":
    file_list = os.listdir(RAW_SNP_PATH)

    with WorkerPool(n_jobs=48) as pool:
        pool.map(getRefPos, file_list, progress_bar=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [05:04<00:00,  1.16it/s]
